In [ ]:
import torch
import torchvision

In [ ]:
class model(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(784, 128)
    self.fc2 = torch.nn.Linear(128, 10)

  def forward(self, x):
    # input shape: 32 * 28 * 28
    x = x.reshape((x.shape[0], 784))
    # output shape: 32*784
    
    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2, train=True)
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x

In [ ]:
device = torch.device("cuda")
# device = torch.device("cpu")

my_model = model()
my_model = my_model.to(device)
my_model.train(True)

model(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# data preprocessing

transform = torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize((0), (1)),
                                # torchvision.transforms.RandomHorizontalFlip(),
                                # torchvision.transforms.
])

dataset = torchvision.datasets.MNIST("./dataset", train=True, download=True, transform=transform)
train_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# hyper parameters

batch_size = 64
epochs = 10
lr = 0.001

In [ ]:
# compile

optimizer = torch.optim.Adam(my_model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
# accuracy

def calc_acc(preds, labels):
  _, pred_max = torch.max(preds, 1)
  acc = torch.sum(pred_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [ ]:
# train

for epoch in range(epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in train_data:
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    # 1-forwarding
    preds = my_model(images)

    # 2-backwarding
    loss = loss_function(preds, labels)
    loss.backward()

    # 3-update
    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data)
  total_acc = train_acc / len(train_data)
  print(f"Epoch: {epoch+1}, Loss: {total_loss}, Accuracy: {total_acc}")

Epoch: 0, Loss: 1.6181116104125977, Accuracy: 0.8743170309168443
Epoch: 1, Loss: 1.5349410772323608, Accuracy: 0.9344183102345416
Epoch: 2, Loss: 1.5203274488449097, Accuracy: 0.9462953091684435
Epoch: 3, Loss: 1.5107839107513428, Accuracy: 0.954974013859275
Epoch: 4, Loss: 1.5047054290771484, Accuracy: 0.960904184434968
Epoch: 5, Loss: 1.5001555681228638, Accuracy: 0.9645189232409381
Epoch: 6, Loss: 1.4966610670089722, Accuracy: 0.9679004530916844
Epoch: 7, Loss: 1.4931107759475708, Accuracy: 0.9711154051172708
Epoch: 8, Loss: 1.491188406944275, Accuracy: 0.9729144456289979
Epoch: 9, Loss: 1.4890328645706177, Accuracy: 0.974646855010661


In [ ]:
# save model

torch.save(model.state_dict(), "mnist.pth")

In [20]:
# inference

import cv2
import numpy as np

# model.tarin(False)
my_model.eval()

# preprocess
img = cv2.imread('5.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))

tensor = transform(img).unsqueeze(0).to(device)
# tensor = tensor.to(device)

# process
preds = my_model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output

5